# Script donnant peut être, un jour, une AFC
A voir si on intègre, si je réussis.

## 1. Chargement des données

In [2]:
import requests
import pandas as pd

## 1.1 Fetching the JSON data
# URL to Nathann Cohen's website. Filters : cabinet, JSON
json_url = "https://jorfsearch.steinertriples.ch/tag/cabinet?format=JSON"

# Fetch the JSON data from the URL
response = requests.get(json_url)
if response.status_code == 200:
    # Save the JSON data to a local file
    with open("nominations.json", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("JSON data downloaded and saved to 'nominations.json'.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")


## 2. Converting it to a working panda
with open("nominations.json", "r", encoding="utf-8") as file:
    data = pd.read_json(file)

nominations = pd.DataFrame(data)
print(nominations.head())

## 1.3 Filtres afin de ne conserver que les cabinets ministériels
print(nominations.columns)

# Unique values and their counts for 'type_ordre'
type_ordre_counts = nominations['type_ordre'].value_counts()
print("Unique values in 'type_ordre' and their counts:")
print(type_ordre_counts)

# Unique values and their counts for 'cabinet_ministeriel'
cabinet_ministeriel_counts = nominations['cabinet_ministeriel'].value_counts()
print("\nUnique values in 'cabinet_ministeriel' and their counts:")
print(cabinet_ministeriel_counts)


nominations_ministerielles = nominations[
    (nominations['cabinet_ministeriel'].isna() != True) &
    (nominations['type_ordre'] == 'nomination')
]

cessations = nominations[
    (nominations['cabinet_ministeriel'].isna() != True) &
    (nominations['type_ordre'] == 'cessation de fonction')
]

# 1.4 Premiers comptes
total_nominations_ministerielles = len(nominations_ministerielles)
total_cessations = len(cessations)

# Convert 'source_date' to datetime
nominations_ministerielles['source_date'] = pd.to_datetime(nominations_ministerielles['source_date'], errors='coerce')
cessations['source_date'] = pd.to_datetime(cessations['source_date'], errors='coerce')

# Counts per year
nominations_ministerielles_per_year = nominations_ministerielles['source_date'].dt.year.value_counts().sort_index()
cessations_per_year = cessations['source_date'].dt.year.value_counts().sort_index()

# Display results
print("Total Nominations Ministerielles:", total_nominations_ministerielles)
print("Total Cessations:", total_cessations)
print("Nominations Ministerielles Per Year:\n", nominations_ministerielles_per_year)
print("Cessations Per Year:\n", cessations_per_year)


## Enrichissons de la couleur politique ! 

import pandas as pd

# Data for French Prime Ministers from 1990 to 2024
data = {
    'PM': [
        'Michel Rocard', 'Édith Cresson', 'Pierre Bérégovoy', 'Édouard Balladur',
        'Alain Juppé', 'Lionel Jospin', 'Jean-Pierre Raffarin', 'Dominique de Villepin',
        'François Fillon', 'Jean-Marc Ayrault', 'Manuel Valls', 'Bernard Cazeneuve',
        'Édouard Philippe', 'Jean Castex', 'Élisabeth Borne', 'Gabriel Attal', 'Michel Barnier'
    ],
    'Start Date': [
        '1988-05-10', '1991-05-15', '1992-04-02', '1993-03-29',
        '1995-05-17', '1997-06-02', '2002-05-06', '2005-05-31',
        '2007-05-17', '2012-05-15', '2014-03-31', '2016-12-06',
        '2017-06-21', '2020-07-03', '2022-05-16', '2024-01-11', '2024-09-05'
    ],
    'End Date': [
        '1991-05-15', '1992-04-02', '1993-03-29', '1995-05-17',
        '1995-11-07', '2002-05-06', '2005-05-31', '2007-05-17',
        '2012-05-15', '2014-03-31', '2016-12-06', '2017-05-15',
        '2020-07-03', '2022-05-16', '2024-01-08', '2024-09-05', None  # None indicates incumbent
    ],
    'Party': [
        'PS', 'PS', 'PS', 'RPR',
        'RPR', 'PS', 'UMP', 'UMP',
        'UMP', 'PS', 'PS', 'PS',
        'LREM', 'LREM', 'LREM', 'LREM', 'LR'
    ],
    'Alignment': [
        'Left', 'Left', 'Left', 'Right',
        'Right', 'Left', 'Right', 'Right',
        'Right', 'Left', 'Left', 'Left',
        'Center-right', 'Center-right', 'Center-right', 'Center-right', 'Right'
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Convert 'Start Date' and 'End Date' to datetime
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])

# Display the DataFrame
print(df)

## On apparie les deux bases

# Ensure 'date_debut' in nominations_ministerielles is in datetime format
nominations_ministerielles['date_debut'] = pd.to_datetime(nominations_ministerielles['date_debut'], errors='coerce')

# Perform a cross join to compare every row in nominations_ministerielles with every row in df
nominations_with_political_alignment = nominations_ministerielles.assign(key=1).merge(
    df.assign(key=1), on='key'
).drop('key', axis=1)

# Filter rows where 'date_debut' falls between 'Start Date' and 'End Date'
nominations_with_political_alignment = nominations_with_political_alignment[
    (nominations_with_political_alignment['date_debut'] >= nominations_with_political_alignment['Start Date']) &
    (nominations_with_political_alignment['date_debut'] <= nominations_with_political_alignment['End Date'])
]

# Drop unnecessary columns and reset index for cleaner results
nominations_with_political_alignment = nominations_with_political_alignment.drop(
    ['Start Date', 'End Date'], axis=1
).reset_index(drop=True)

# Display the merged DataFrame
print(nominations_with_political_alignment)



JSON data downloaded and saved to 'nominations.json'.
                                       organisations source_date  \
0  [{'nom': 'Préfecture du Vaucluse', 'wikidata_i...  2024-12-18   
1  [{'nom': 'Préfecture du Vaucluse', 'wikidata_i...  2024-12-18   
2  [{'nom': 'Cabinet du Premier ministre', 'wikid...  2024-12-18   
3  [{'nom': 'Cabinet du Premier ministre', 'wikid...  2024-12-18   
4  [{'nom': 'Cabinet du Premier ministre', 'wikid...  2024-12-18   

              source_id source_name               type_ordre sous-prefet  \
0  JORFTEXT000050789652        JORF               nomination        true   
1  JORFTEXT000050789650        JORF    cessation de fonction         NaN   
2  JORFTEXT000050789616        JORF               nomination         NaN   
3  JORFTEXT000050789264        JORF  délégation de signature         NaN   
4  JORFTEXT000050789264        JORF  délégation de signature         NaN   

                                 cabinet sexe           nom           prenom  \


/tmp/ipykernel_62591/2233039059.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nominations_ministerielles['source_date'] = pd.to_datetime(nominations_ministerielles['source_date'], errors='coerce')
/tmp/ipykernel_62591/2233039059.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cessations['source_date'] = pd.to_datetime(cessations['source_date'], errors='coerce')
/tmp/ipykernel_62591/2233039059.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

     organisations source_date             source_id source_name  type_ordre  \
0               []  2024-08-30  JORFTEXT000050148223        JORF  nomination   
1               []  2024-08-24  JORFTEXT000050133926        JORF  nomination   
2               []  2024-07-31  JORFTEXT000050060447        JORF  nomination   
3               []  2024-07-18  JORFTEXT000050011569        JORF  nomination   
4               []  2024-07-13  JORFTEXT000049963270        JORF  nomination   
...            ...         ...                   ...         ...         ...   
4483            []  1990-05-23  JORFTEXT000000159814        JORF  nomination   
4484            []  1990-05-23  JORFTEXT000000159814        JORF  nomination   
4485            []  1990-05-15  JORFTEXT000000708891        JORF  nomination   
4486            []  1990-04-20  JORFTEXT000000349222        JORF  nomination   
4487            []  1990-03-24  JORFTEXT000000349503        JORF  nomination   

     sous-prefet                       

In [ ]:
!pip install openpyxl

import pandas as pd

# Load the Excel file
file_path = "tab_mentions.xlsx"  # Ensure the file is in the same directory or provide the full path
data = pd.read_excel(file_path)

# Display the first few rows
print(data.head())
print(data.shape[0])

## J'ATTENDS LES SCORES DEJA FAITS


   Unnamed: 0   prenom       academie      ville             mention
0           0  kamilya  aix-marseille   tarascon  Admis mention Bien
1           1   meyssa  aix-marseille        apt    Admis mention TB
2           2   anasse  aix-marseille  cavaillon    Admis mention AB
3           3     yaël  aix-marseille  vitrolles              Admis 
4           4    imane  aix-marseille      arles    Admis mention AB
50530
